Quora duplicate questions dataset

In [1]:
import pandas as pd

df = pd.read_csv('quora_questions.csv')

print(df.shape)

(404351, 6)


In [2]:
import pandas as pd

df = pd.read_csv('quora_questions.csv')

df = df.sample(n=40000,random_state=42)

df.head()

,id,qid1,qid2,question1,question2,is_duplicate
120567,120567,238932,238933,How does the Boggart work?,What would the boggart of a boggart be?,0
324466,324466,636476,636477,What is difference between project manager and...,What are the differences between project manag...,0
398558,398558,778728,778729,What hotel in Jabalpur would be safe for unmar...,What hotel in Allahabad would be safe for unma...,0
339914,339914,666314,666315,What is stronger - Super Saiyan 4 or Super Sai...,How does Gohan turn into Super Saiyan 2?,0
185732,185732,366764,366765,How do I fill in Address Line 1 and Address Li...,How do I register desired web address?,0


In [3]:
q1 = df['question1'].values.tolist()
q2 = df['question2'].values.tolist()

In [4]:
print(df.duplicated().sum())
print(df.isnull().sum())

0
id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64


In [5]:
# remove duplicated and null rows

df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

In [6]:
# see data distribution

val = df['is_duplicate'].value_counts()
zc=val[0]
oc=val[1]
print(f"Percentage of 0: {zc*100/(zc+oc):.2f}%")
print(f"Percentage of 1: {oc*100/(zc+oc):.2f}%")

Percentage of 0: 63.06%
Percentage of 1: 36.94%


In [7]:
X = df[['question1','question2']].values
Y = df['is_duplicate'].values

In [8]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [9]:
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
import warnings

warnings.filterwarnings('ignore')

stop_words = set(stopwords.words('english'))

def remove_html_tags(text):
    """Remove HTML tags using BeautifulSoup"""
    if not isinstance(text, str) or not text:
        return text
    
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

exclude = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

def remove_punctuation(s:str)->str:
    return s.translate(str.maketrans('','',exclude))

def remove_url(s: str) -> str:
    """Remove URLs from string - simple and effective"""
    if not isinstance(s, str) or not s:
        return s
    
    # Pattern that catches most URLs
    url_pattern = r'http[s]?://\S+|www\.\S+|\S+\.(com|org|net|edu|gov|io|co)\S*'
    
    # Remove URLs
    clean_text = re.sub(url_pattern, '', s, flags=re.IGNORECASE)
    
    # Clean up spaces
    return ' '.join(clean_text.split()) 

def pre_process_paragraph(s:str)->list[str]:
    # lowercase and stripping
    s = s.strip().lower()

    # removing urls, punctuations and html tags + [math] substrings
    s = re.sub(r'\[math\].*?\[/math\]', '', s)
    s = remove_url(remove_punctuation(remove_html_tags(s)))

    # Clean up extra whitespace
    s = ' '.join(s.split())

    # word level tokenize
    tokenized_para = word_tokenize(s)

    # removing stopwords
    tokenized_para = [ele for ele in tokenized_para if ele not in stop_words]

    # spell checker (skip for performance)

    # stemming (skip for performance)

    return tokenized_para

In [10]:
corpus = [pre_process_paragraph(q1) for q1,_ in x_train]
corpus.extend([pre_process_paragraph(q2) for _,q2 in x_train])

In [11]:
corpus.__len__()

64000

In [16]:
import gensim

model = gensim.models.Word2Vec(workers=6,window=3,vector_size=300)

In [17]:
model.build_vocab(corpus)

In [18]:
model.train(corpus,epochs=model.epochs,total_examples=model.corpus_count)

(1528852, 1774560)

In [21]:
# save word2vec model
import joblib

joblib.dump(model,'models\\w2v_duplicate_.pkl')

['models\\w2v_duplicate_.pkl']

In [23]:
import fuzzywuzzy.fuzz as fuzz
import joblib

model = joblib.load('models\\w2v_duplicate_.pkl')

def word_level_w2v(s):
    """Get word vector with error handling"""
    try:
        return model.wv[s]
    except KeyError:
        # Return zero vector if word not in vocabulary
        return np.zeros(model.wv.vector_size)

def getCommonWords(s1:str,s2:str):
    st1 = set(s1.split())
    st2 = set(s2.split())
    
    return len(st1 & st2)

def getCommonStopwords(s1:str,s2:str):
    words1 = set(s1.split())
    words2 = set(s2.split())

    return len(words1 & stop_words & words2)

def getTotalUniqueWords(s1:str,s2:str):
    st1 = set(s1.split())
    st2 = set(s2.split())
    
    return len(st1) + len(st2)

def getStopwordsCount(s1:str):
    return len(stop_words & set(s1.split()))

def getLongestSubstringRatio(s1: str, s2: str) -> float:
    """Optimized longest common substring ratio calculation"""
    if not s1 or not s2:
        return 0.0
    
    s1, s2 = s1.lower(), s2.lower()
    m, n = len(s1), len(s2)
    
    # Use rolling arrays to save memory
    prev = [0] * (n + 1)
    curr = [0] * (n + 1)
    
    max_length = 0
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if s1[i-1] == s2[j-1]:
                curr[j] = prev[j-1] + 1
                max_length = max(max_length, curr[j])
            else:
                curr[j] = 0
        
        # Swap arrays
        prev, curr = curr, prev
    
    min_length = min(m, n)
    return max_length / min_length if min_length > 0 else 0.0

def getFuzzyMetrics(s1,s2):
    # fuzz ratio
    fuzz_ratio = fuzz.ratio(s1,s2)

    # fuzz_partial_ratio
    fuzz_partial_ratio = fuzz.partial_ratio(s1,s2)

    # fuzz_sort_ratio
    fuzz_set_ratio = fuzz.token_sort_ratio(s1,s2)

    # fuzz set ratio
    fuzz_token_ratio = fuzz.token_set_ratio(s1,s2)

    return fuzz_ratio,fuzz_partial_ratio,fuzz_set_ratio,fuzz_token_ratio

def w2v_sentence(s1:str,s2:str):
    tokens_s1 = pre_process_paragraph(s1)
    tokens_s2 = pre_process_paragraph(s2)

    # 1. self engineered features

    n_chars1 = len(s1)
    n_chars2 = len(s2)
    n_words1 = len(s1.split())
    n_words2 = len(s2.split())
    common_words = getCommonWords(s1,s2)
    total_unique_words = getTotalUniqueWords(s1,s2)
    common_ratio = common_words / total_unique_words
    common_stopwords = getCommonStopwords(s1,s2)
    stopwords1 = getStopwordsCount(s1)
    stopwords2 = getStopwordsCount(s2)
    common_tokens = len(set(tokens_s1) & set(tokens_s2))
    tokens1 = len(set(tokens_s1))
    tokens2 = len(set(tokens_s2))

    # 1.1 token level features
    cwc_min = common_words/min(n_words1,n_words2)
    cwc_max = common_words/max(n_words1,n_words2)
    csc_min = common_stopwords/min(stopwords1,stopwords2) if min(stopwords1,stopwords2) else 0.0
    csc_max = common_stopwords/max(stopwords1,stopwords2) if max(stopwords1,stopwords2) else 0.0
    ctc_min = common_tokens/min(tokens1,tokens2) if min(tokens1,tokens2) else 0.0
    ctc_max = common_tokens/max(tokens1,tokens2) if max(tokens1,tokens2) else 0.0
    last_word_eq = bool(s1.split()[-1]==s2.split()[-1])
    first_word_eq = bool(s1.split()[0]==s2.split()[0])

    mean_length = (n_words1 + n_words2) / 2
    abs_len_diff = abs(n_words2 - n_words1)
    longest_substring_ratio = getLongestSubstringRatio(s1,s2)

    fuzz_ratio,fuzz_partial_ratio,fuzz_set_ratio,fuzz_token_ratio = getFuzzyMetrics(s1,s2)

    v1 = [word_level_w2v(ele) for ele in tokens_s1] or [np.zeros(model.wv.vector_size)]
    v2 = [word_level_w2v(ele) for ele in tokens_s2] or [np.zeros(model.wv.vector_size)]

    # additional features vector
    additional_vec = [n_chars1,n_chars2,n_words1,n_words2,common_words,total_unique_words,common_ratio]
    additional_vec.extend([cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq])
    additional_vec.extend([mean_length,abs_len_diff,longest_substring_ratio])
    additional_vec.extend([fuzz_ratio,fuzz_partial_ratio,fuzz_set_ratio,fuzz_token_ratio])

    final_feature_vector = np.mean(np.concatenate([v1,v2]),axis=0)
    final_feature_vector = np.concatenate([final_feature_vector,additional_vec])

    return final_feature_vector

In [24]:
getCommonStopwords('this is akash','this here is akash and papa')

2

In [25]:
w2v_sentence('i want to become ias','how do i become an ias')[-10:]

array([ 0.66666667,  1.        ,  0.        ,  5.5       ,  1.        ,
        0.4       , 67.        , 70.        , 57.        , 76.        ])

In [26]:
x_train_real = [w2v_sentence(s1,s2) for s1,s2 in x_train]
x_test_real = [w2v_sentence(s1,s2) for s1,s2 in x_test]

In [46]:
from xgboost import XGBClassifier

xgb = XGBClassifier(
    n_estimators=300,
    max_depth=20,              
    min_samples_split=10,       
    min_samples_leaf=5,           
    random_state=42,                 
    class_weight='balanced'
)

xgb.fit(x_train_real,y_train)

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [47]:
import joblib

joblib.dump(xgb,'models//xgb_model.pkl')

['models//xgb_model.pkl']

In [48]:
# predictions
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

xgb_model = joblib.load('models\\xgb_model.pkl')

y_pred = xgb_model.predict(x_test_real)

print(f"Accuracy: {accuracy_score(y_pred=y_pred,y_true=y_test):.4f}")
print(f"Precision: {precision_score(y_pred=y_pred,y_true=y_test):.4f}")
print(f"Recall: {recall_score(y_pred=y_pred,y_true=y_test):.4f}")
print(f"F1 score: {f1_score(y_pred=y_pred,y_true=y_test):.4f}")

Accuracy: 0.7957
Precision: 0.7217
Recall: 0.7300
F1 score: 0.7258


In [49]:
import joblib
from time import time

start = time()

xgb_model = joblib.load('models\\xgb_model.pkl')

s1 = 'where do i learn data science'
s2 = 'resources to get started with data science'

sample1 = w2v_sentence(s1,s2)

y_pred = xgb_model.predict([sample1])
print("Duplicate" if y_pred==1 else 'Non Duplicate')
print(f"Prediction took {time()-start:.4f} seconds")

Non Duplicate
Prediction took 0.0521 seconds


# Model using PyTorch

In [76]:
import pandas as pd

# df = pd.read_csv("quora_questions.csv").sample(n=100000,random_state=42)
df = pd.read_csv("quora_questions.csv")

# Drop rows with missing values BEFORE splitting
df = df.dropna(subset=['question1', 'question2'])

X = df[['question1','question2']].values
Y = df['is_duplicate'].values

In [77]:
from sklearn.model_selection import train_test_split

x_train,x_test_val,y_train,y_test_val = train_test_split(X,Y,test_size=0.3,random_state=42)
x_val,x_test,y_val,y_test = train_test_split(x_test_val,y_test_val,test_size=0.5,random_state=42)

In [78]:
print(f"Number of unique elements in corpus: {len(set(tuple(x) for x in corpus))}")

Number of unique elements in corpus: 56841


In [79]:
vocab = {'<UNK>':0}
for tokenized_sentence in corpus:
    for word in tokenized_sentence:
        if word not in vocab:
            vocab[word] = len(vocab)

In [80]:
def para_to_indices(s:str):
    arr = []
    for word in pre_process_paragraph(s):
        arr.append(0 if word not in vocab else vocab[word])
    return arr

In [81]:
import numpy as np

x_train = [np.concatenate([para_to_indices(sen1),para_to_indices(sen2)]) for sen1,sen2 in x_train]
x_val = [np.concatenate([para_to_indices(sen1),para_to_indices(sen2)]) for sen1,sen2 in x_val]
x_test = [np.concatenate([para_to_indices(sen1),para_to_indices(sen2)]) for sen1,sen2 in x_test]

In [82]:
from torch.utils.data import Dataset,DataLoader
import torch

class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        # self.x[index] is already a concatenated numpy array of indices
        return torch.tensor(self.x[index], dtype=torch.long), torch.tensor(self.y[index], dtype=torch.float32)

In [83]:
train_dataset = CustomDataset(x_train,y_train)
val_dataset = CustomDataset(x_val,y_val)
test_dataset = CustomDataset(x_test,y_test)

In [84]:
import torch
from torch.nn.utils.rnn import pad_sequence

torch.manual_seed(42)

def collate_pad(batch):
    """
    Collate function to pad sequences in a batch to the same length.
    Args:
        batch: list of (tensor, label) tuples
    Returns:
        padded_tensors: tensor of shape (batch_size, max_seq_len)
        labels: tensor of shape (batch_size,)
    """
    sequences, labels = zip(*batch)
    # Pad sequences to the length of the longest sequence in the batch
    padded_tensors = pad_sequence(sequences, batch_first=True, padding_value=0) # type:ignore
    labels = torch.stack(labels)
    return padded_tensors, labels

train_loader = DataLoader(train_dataset,batch_size=8,collate_fn=collate_pad)
val_loader = DataLoader(val_dataset,batch_size=16,shuffle=True,collate_fn=collate_pad)
test_loader = DataLoader(test_dataset,batch_size=16,shuffle=True,collate_fn=collate_pad)

In [85]:
from torch import nn

class MyNN(nn.Module):
    def __init__(self, embedding_dim=150, dropout=0.3, hidden_size=128):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=len(vocab), embedding_dim=embedding_dim)
        self.lstm = nn.LSTM(
            embedding_dim,
            num_layers=4,
            dropout=dropout,
            hidden_size=hidden_size,
            batch_first=True,
            bidirectional=True
        )
        self.dropout = nn.Dropout(p=dropout)
        self.linear = nn.Linear(hidden_size * 2, 1)  # hidden_size * 2 for bidirectional

    def forward(self, x):
        embedding = self.embedding(x)
        _, (hd, _) = self.lstm(embedding)
        # hd shape: (num_layers * num_directions, batch, hidden_size)
        # For bidirectional, concatenate last layer's forward and backward hidden states
        out = torch.cat((hd[-2], hd[-1]), dim=1)  # shape: (batch, hidden_size*2)
        out = self.dropout(out)
        return self.linear(out).squeeze(-1)

In [86]:
# from torch import nn

# class MyNN(nn.Module):
#     def __init__(self, embedding_dim=256, num_heads=4, num_layers=4, dropout=0.3):
#         super().__init__()
#         self.embedding = nn.Embedding(len(vocab), embedding_dim)
#         encoder_layer = nn.TransformerEncoderLayer(
#             d_model=embedding_dim,
#             nhead=num_heads,
#             dropout=dropout,
#             batch_first=True
#         )
#         self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
#         self.linear = nn.Linear(embedding_dim, 1)

#     def forward(self, x):
#         emb = self.embedding(x)
#         out = self.transformer(emb)
#         out = out.mean(dim=1)  # Pooling
#         return self.linear(out).squeeze(-1)


In [87]:
neta = 2e-4
lambda_ = 3e-4
epochs = 50

In [88]:
# model and device
model = MyNN()
device = torch.device('cuda')
model.to(device)

# optimizer
optimizer = torch.optim.Adam(model.parameters(),lr=neta,weight_decay=lambda_)

# loss fn
loss_fn = nn.BCEWithLogitsLoss()

In [89]:
import torch
from tqdm import tqdm

torch.manual_seed(42)

# Early stopping parameters
patience = 3  # Number of epochs to wait for improvement
best_val_loss = float('inf')
patience_counter = 0
best_model_state = None

# Training loop with early stopping
for epoch in range(epochs):
    # Training phase
    model.train()
    total_loss = 0.0
    
    # Add progress bar for training
    train_pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs} [Train]', leave=False)
    for train_data, train_label in train_pbar:
        # Move to device
        train_data, train_label = train_data.to(device), train_label.to(device)

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        out = model(train_data)

        # Loss calculation
        loss = loss_fn(out, train_label)

        # Backward pass
        loss.backward()

        # Update gradients
        optimizer.step()

        total_loss += loss.item()
        
        # Update progress bar with current loss
        train_pbar.set_postfix({'loss': f'{loss.item():.4f}'})

    avg_train_loss = total_loss / len(train_loader)
    
    # Validation phase
    model.eval()
    val_loss = 0.0
    
    # Add progress bar for validation
    val_pbar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{epochs} [Val]', leave=False)
    with torch.no_grad():
        for val_data, val_label in val_pbar:
            val_data, val_label = val_data.to(device), val_label.to(device)
            val_out = model(val_data)
            batch_loss = loss_fn(val_out, val_label).item()
            val_loss += batch_loss
            
            # Update progress bar with current loss
            val_pbar.set_postfix({'loss': f'{batch_loss:.4f}'})
    
    avg_val_loss = val_loss / len(val_loader)
    
    print(f"Epoch: {epoch+1} ====> Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
    
    # Early stopping check
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        # Save best model state
        best_model_state = model.state_dict().copy()
        print(f"✅ Validation loss improved! Saving best model...")
    else:
        patience_counter += 1
        print(f"⚠️  No improvement. Patience: {patience_counter}/{patience}")
        
        if patience_counter >= patience:
            print(f"🛑 Early stopping triggered after {epoch+1} epochs!")
            # Load best model state
            model.load_state_dict(best_model_state)
            break

print(f"\n✅ Training completed! Best validation loss: {best_val_loss:.4f}")

Epoch: 1 ====> Train Loss: 0.5478 | Val Loss: 0.5239
✅ Validation loss improved! Saving best model...


Epoch: 2 ====> Train Loss: 0.5264 | Val Loss: 0.5175
✅ Validation loss improved! Saving best model...


Epoch: 3 ====> Train Loss: 0.5212 | Val Loss: 0.5129
✅ Validation loss improved! Saving best model...


Epoch: 4 ====> Train Loss: 0.5171 | Val Loss: 0.5098
✅ Validation loss improved! Saving best model...


Epoch: 5 ====> Train Loss: 0.5129 | Val Loss: 0.5074
✅ Validation loss improved! Saving best model...


...

Epoch: 20 ====> Train Loss: 0.4894 | Val Loss: 0.4978
⚠️  No improvement. Patience: 1/3


Epoch: 21 ====> Train Loss: 0.4886 | Val Loss: 0.4968
✅ Validation loss improved! Saving best model...


Epoch: 22 ====> Train Loss: 0.4878 | Val Loss: 0.4977
⚠️  No improvement. Patience: 1/3


Epoch: 23 ====> Train Loss: 0.4870 | Val Loss: 0.4984
⚠️  No improvement. Patience: 2/3


Epoch: 24 ====> Train Loss: 0.4861 | Val Loss: 0.4985
⚠️  No improvement. Patience: 3/3
🛑 Early stopping triggered after 24 epochs!

✅ Training completed! Best validation loss: 0.4968


In [90]:
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

torch.manual_seed(42)

# Fixed evaluation function
def evaluate_rnn_model():
    model.eval()
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for test_batch, test_labels in test_loader:
            test_batch, test_labels = test_batch.to(device), test_labels.to(device)

            y_pred = model(test_batch)
            
            # Fixed prediction calculation for sigmoid output
            predictions = (torch.sigmoid(y_pred) > 0.5).float()
            
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(test_labels.cpu().numpy())

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions, zero_division=0)
    recall = recall_score(all_labels, all_predictions, zero_division=0)
    f1 = f1_score(all_labels, all_predictions, zero_division=0)
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

# Run evaluation
print("📊 Evaluating model on testing data...")
evaluate_rnn_model()

📊 Evaluating model on testing data...


{'accuracy': 0.7578025819003182,
 'precision': 0.7339561580265006,
 'recall': 0.5377354270120341,
 'f1_score': 0.6207074619158275}

# Using pre-trained bert model

In [1]:
NROWS = 200000

In [2]:
from torch.utils.data import Dataset
from transformers import BertTokenizer
import warnings

warnings.filterwarnings("ignore")

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Reload original data for BERT
from sklearn.model_selection import train_test_split
import pandas as pd

df = pd.read_csv('quora_questions.csv', nrows=NROWS)

# Drop rows with missing values
df.dropna(subset=['question1', 'question2', 'is_duplicate'], inplace=True)

X = df[['question1','question2']].values
Y = df['is_duplicate'].values

# Split: 70% train, 15% validation, 15% test
x_train_bert, x_temp, y_train_bert, y_temp = train_test_split(X, Y, test_size=0.3, random_state=42)
x_val_bert, x_test_bert, y_val_bert, y_test_bert = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

# Extract questions and convert to strings
train_q1 = [str(pair[0]) for pair in x_train_bert]
train_q2 = [str(pair[1]) for pair in x_train_bert] 
val_q1 = [str(pair[0]) for pair in x_val_bert]
val_q2 = [str(pair[1]) for pair in x_val_bert]
test_q1 = [str(pair[0]) for pair in x_test_bert]
test_q2 = [str(pair[1]) for pair in x_test_bert]

# Create encodings and datasets
train_encodings = tokenizer(train_q1, train_q2, padding=True, truncation=True, max_length=128, return_tensors='pt')
train_dataset = CustomDataset(train_encodings, y_train_bert)

val_encodings = tokenizer(val_q1, val_q2, padding=True, truncation=True, max_length=128, return_tensors='pt')
val_dataset = CustomDataset(val_encodings, y_val_bert)

test_encodings = tokenizer(test_q1, test_q2, padding=True, truncation=True, max_length=128, return_tensors='pt')
test_dataset = CustomDataset(test_encodings, y_test_bert)

print(f"Train size: {len(train_dataset)}")
print(f"Validation size: {len(val_dataset)}")
print(f"Test size: {len(test_dataset)}")

c:\Users\91748\OneDrive\Desktop\duplicate question classifier\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are

Train size: 139999
Validation size: 30000
Test size: 30000


In [3]:
len(test_dataset)

30000

In [6]:
from transformers.trainer import Trainer
from transformers.training_args import TrainingArguments
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
from transformers import BertForSequenceClassification
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model = model.to(device)  # 🎯 Move model to GPU

def compute_metrics(eval_pred):
    logits, labels = eval_pred.predictions, eval_pred.label_ids
    # For binary classification with 2 labels
    preds = logits.argmax(-1)  # Get predicted class (0 or 1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels,preds)
    return {"accuracy": acc, 'f1_score':f1, "precision":precision,"recall":recall}

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,          # 2-4 is guud
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    learning_rate=1e-5,          # Lower LR
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # Your processed train CustomDataset
    eval_dataset=val_dataset,    # Your processed test CustomDataset
    compute_metrics=compute_metrics
)

Using device: cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1 Score,Precision,Recall
1,0.344500,0.330446,0.869433,0.834131,0.787227,0.886978
2,0.311700,0.392492,0.878833,0.841564,0.815440,0.869416
3,0.235900,0.475074,0.883767,0.847269,0.824763,0.871037


TrainOutput(global_step=52500, training_loss=0.308428998819987, metrics={'train_runtime': 26783.1862, 'train_samples_per_second': 15.681, 'train_steps_per_second': 1.96, 'total_flos': 2.762646347950848e+16, 'train_loss': 0.308428998819987, 'epoch': 3.0})

In [ ]:
trainer.train(resume_from_checkpoint='results/checkpoint-10000')

In [8]:
# Final testing on pristine testing dataset

import os,re

# Find the latest checkpoint automatically
results_dir = './results'
checkpoints = [d for d in os.listdir(results_dir) if d.startswith('checkpoint-')]

if checkpoints:
    # Extract checkpoint numbers and find the maximum
    checkpoint_numbers = [int(re.search(r'checkpoint-(\d+)', cp).group(1)) for cp in checkpoints] # type:ignore
    latest_checkpoint_num = max(checkpoint_numbers)
    model_path = f'{results_dir}/checkpoint-{latest_checkpoint_num}'
    print(f"Loading latest checkpoint: {model_path}")
else:
    raise FileNotFoundError("No checkpoints found in ./results directory")

model = BertForSequenceClassification.from_pretrained(model_path)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # Your processed train CustomDataset
    eval_dataset=test_dataset,    # Your processed test CustomDataset
    compute_metrics=compute_metrics
)

def evaluate_bert_model(trainer):
    results = trainer.evaluate()
    print(f"Test Loss: {results['eval_loss']:.4f}")
    print(f"Test Accuracy: {results['eval_accuracy']*100:.2f}%" if 'eval_accuracy' in results else "Accuracy: NA")
    print(f"Precision: {results['eval_precision']*100:.2f}%" if 'eval_precision' in results else "Precision: NA")
    print(f"Recall: {results['eval_recall']*100:.2f}%" if 'eval_recall' in results else "Recall: NA")
    print(f"F1 Score: {results['eval_f1_score']*100:.2f}%" if 'eval_f1_score' in results else "F1 Score: NA")

evaluate_bert_model(trainer)

Loading latest checkpoint: ./results/checkpoint-52500


Test Loss: 0.4768
Test Accuracy: 88.21%
Precision: 81.84%
Recall: 87.28%
F1 Score: 84.47%


# Real-world usage of BERT fine-tuned solution

In [11]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import os
import re

# Find the latest checkpoint automatically
results_dir = './results'
checkpoints = [d for d in os.listdir(results_dir) if d.startswith('checkpoint-')]

if checkpoints:
    # Extract checkpoint numbers and find the maximum
    checkpoint_numbers = [int(re.search(r'checkpoint-(\d+)', cp).group(1)) for cp in checkpoints] # type:ignore
    latest_checkpoint_num = max(checkpoint_numbers)
    model_path = f'{results_dir}/checkpoint-{latest_checkpoint_num}'
    print(f"Loading latest checkpoint: {model_path}")
else:
    raise FileNotFoundError("No checkpoints found in ./results directory")

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained(model_path)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

Loading latest checkpoint: ./results/checkpoint-10000


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [12]:
import torch.nn.functional as F

def predict_duplicate(question1, question2):
    """
    Predict if two questions are duplicates using the trained BERT model
    
    Args:
        question1 (str): First question
        question2 (str): Second question  
        model_path (str): Path to saved model checkpoint
        
    Returns:
        dict: Prediction results with probability and classification
    """
    
    # Tokenize input
    inputs = tokenizer(
        question1, 
        question2,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors='pt'
    )
    
    # Move to device
    inputs = {key: val.to(device) for key, val in inputs.items()}
    
    # Make prediction
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        
        # Get probabilities
        probabilities = F.softmax(logits, dim=-1)
        
        # Probability of being duplicate (class 1)
        duplicate_prob = probabilities[0][1].item()
    
    return {
        'duplicate_probability': duplicate_prob
    }

In [13]:
# Example usage:
result = predict_duplicate(
    "what is the process of coming late?",
    "what is the process of coming on time?"
)

print(f"Duplicate Probability: {result['duplicate_probability']:.4f}")

Duplicate Probability: 0.2776
